In [29]:
pip install nl2query azure-kusto-data

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [31]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from nl2query import KustoQuery

In [3]:
AAD_TENANT_ID = "0eadb77e-42dc-47f8-bbe3-ec2395e0712c"
KUSTO_CLUSTER = "https://help.kusto.windows.net/"


In [18]:
def authenticate_toCluster(clusterName, tenantID):
    """Authenticates to the cluster using AAD device code flow"""
    KCSB = KustoConnectionStringBuilder.with_aad_device_authentication(
        clusterName)
    KCSB.authority_id = tenantID
    return KustoClient(KCSB)


In [19]:
KUSTO_CLIENT = authenticate_toCluster(KUSTO_CLUSTER, AAD_TENANT_ID)

In [33]:
def queryGenerator(cols, tableName, prompt):
    queryfier = KustoQuery(cols, tableName)
    return queryfier.generate_query(prompt)

In [34]:
def queryExecute(database, query):
    return KUSTO_CLIENT.execute(database, query)

In [35]:
def queryResponse(response):
    return dataframe_from_result_table(response.primary_results[0])


In [47]:
def getDatabase():
    database = queryExecute("", ".show databases | project DatabaseName")
    return queryResponse(database)

In [48]:
def getTables(database):
    tables = queryExecute(database, ".show tables | project TableName")
    return queryResponse(tables)

In [50]:
def getColumns(database, table):
    columns = queryExecute(database, table + " | getschema | project ColumnName")
    return queryResponse(columns)

In [36]:
response = queryExecute("Samples", "StormEvents | take 10")
print(queryResponse(response))

                  StartTime                   EndTime  EpisodeId  EventId  \
0 2007-09-29 08:11:00+00:00 2007-09-29 08:11:00+00:00      11091    61032   
1 2007-09-18 20:00:00+00:00 2007-09-19 18:00:00+00:00      11074    60904   
2 2007-09-20 21:57:00+00:00 2007-09-20 22:05:00+00:00      11078    60913   
3 2007-12-30 16:00:00+00:00 2007-12-30 16:05:00+00:00      11749    64588   
4 2007-12-20 07:50:00+00:00 2007-12-20 07:53:00+00:00      12554    68796   
5 2007-12-20 10:32:00+00:00 2007-12-20 10:36:00+00:00      12554    68814   
6 2007-12-20 08:47:00+00:00 2007-12-20 08:48:00+00:00      12554    68834   
7 2007-12-28 02:03:00+00:00 2007-12-28 02:11:00+00:00      12561    68846   
8 2007-12-07 14:00:00+00:00 2007-12-08 04:00:00+00:00      13183    73241   
9 2007-12-13 09:02:00+00:00 2007-12-13 10:30:00+00:00      11780    64725   

            State          EventType  InjuriesDirect  InjuriesIndirect  \
0  ATLANTIC SOUTH         Waterspout               0                 0   
1   

In [25]:
response = queryExecute("FindMyPartner", 'Partner | where Contact == "info@dataCore.co.il" | project Partner')
print(queryResponse(response))

           Partner
0  Datacore IL Ltd


In [28]:
response = queryExecute("ContosoSales", 'SalesFact | where ProductKey == 2023 | summarize avg(SalesAmount) by ProductKey | top 5 by avg_SalesAmount desc | project ProductKey')
print(queryResponse(response))

   ProductKey
0        2023


In [38]:
query = queryGenerator(["ProductKey", "SalesAmount"], "SalesFact", "What is the average sales amount for product 2023?")
response = queryExecute("ContosoSales",query)
print(queryResponse(response))

/Users/parthunallapu/Library/Python/3.9/lib/python/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


   avg_SalesAmount
0        89.330689
